In [39]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import smart_resize
from tensorflow.keras import backend as K
from tensorflow.keras.utils import image_dataset_from_directory, load_img, img_to_array, array_to_img, plot_model
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import json
from functools import partial

In [43]:
# Загрузка модели
pattern = load_model('path')

# Загрузка весов модели
pattern.load_weights('car.h5')

In [44]:
# импортируем карту парковки
parking_regions = json.load(open('parking_map.json'))['regions']
def getSquares(region):
    all_x = region['shape_attributes']['all_points_x']
    all_y = region['shape_attributes']['all_points_y']
    return {'x1': min(all_x), 'x2': max(all_x), 'y1': min(all_y), 'y2': max(all_y)}

regions = list(map(getSquares, parking_regions))

In [45]:
import cv2

def getImage(img, target_size, region):
        cut_image = img[region['y1']:region['y2'], region['x1']:region['x2'], : ]
        return smart_resize(cut_image, target_size)

# адрес потока камеры    
camera_url = 'http://96.56.250.139:8200/mjpg/video.mjpg'
cap = cv2.VideoCapture(camera_url)

while True:
    # получаем снимок с потока
    ret, image = cap.read()
    
    image_arr = img_to_array(image)   

    # нарезаем входное изображение согласно карте парковки
    parking_images = np.array(list(map(
        partial(getImage, image_arr, (image_height, image_width)), 
        regions
    )))
    
    # распознаем отдельно парковочные места
    predictions = pattern.predict(parking_images)

    empties = 0
    for i, pred in enumerate(predictions): 
        # считаем свободные места
        if pred[0] <= 0.5: 
            empties += 1

        # создаем прямоугольник
        start = (regions[i]['x1'], regions[i]['y1'])
        width = regions[i]['x2'] - regions[i]['x1']
        height = regions[i]['y2'] - regions[i]['y1']
        
        # в формате BGR 
        color = (0, 0, 255) if pred[0] >= 0.5 else (0, 255, 0)
        
        image = cv2.rectangle(
            image, 
            (regions[i]['x1'], regions[i]['y1']), 
            (regions[i]['x2'], regions[i]['y2']), 
            color, 
            1
        )
        

    # пишем кол-во свободных парковочных мест   
    image = cv2.putText(
        image, '{}/{}'.format(empties, len(regions)), 
        (1000, 600),                # point
        cv2.FONT_HERSHEY_SIMPLEX,   # font
        1,                          # font scale 
        (0, 255, 0),                # color
        1                           # thickness
    )

    # выводим на экран
    cv2.imshow('Parking camera (Esc for exit)', image) 
    
    # если нажат Esc, то прерываем работу и закрываем окно cv2
    if cv2.waitKey(500) == 27:
        cv2.destroyAllWindows()
        break
        